# 01_import dataset

## 01-1_import data

In [1]:
import os
import json
import pandas as pd

#https://drive.google.com/drive/folders/18qV82fNY3IIWu3BRoGqm_LNgJzE8Akbr?usp=drive_link
#base_dir = "/Users/Andypon/10_交大研究所/1141_01_機器學習與金融科技/data"
base_dir= '/Users/andyw.p.chen/Documents/Project/datasets'
#base_dir=  "c:\Users\user\Downloads\datasets"

def load_json_to_df(filename: str) -> pd.DataFrame:
    file_path = os.path.join(base_dir, filename)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 如果是 { "target": {id: value, ...} }
    if isinstance(data, dict) and len(data) == 1 and isinstance(next(iter(data.values())), dict):
        key, inner = next(iter(data.items()))
        return pd.DataFrame(list(inner.items()), columns=["id", key])

    # dict of scalar
    if isinstance(data, dict):
        return pd.DataFrame([{"code": k, "desc": v} for k, v in data.items()])

    # list of dict
    elif isinstance(data, list):
        return pd.DataFrame(data)

    else:
        raise ValueError(f"Unsupported JSON structure in {filename}: {type(data)}")


def load_csv_to_df(filename: str) -> pd.DataFrame:
    """讀取 CSV 並轉為 DataFrame。"""
    return pd.read_csv(os.path.join(base_dir, filename))

# JSON 資料
##mcc_codes_df = load_json_to_df("mcc_codes.json")
train_fraud_labels_df = load_json_to_df("train_fraud_labels.json")

# CSV 資料
cards_df = load_csv_to_df("cards_data.csv")
transactions_df = load_csv_to_df("transactions_data.csv")
users_df = load_csv_to_df("users_data.csv")

# 簡單檢查
#print(mcc_codes_df.head())
#print(train_fraud_labels_df.head())
#print(cards_df.head())
#print(transactions_df.head())
#print(users_df.apthead())


## 01-2_rename variable in each data set

In [2]:
train_fraud_labels_df = train_fraud_labels_df.rename(columns={'id': 'transactions_id'})
train_fraud_labels_df = train_fraud_labels_df.rename(columns={'target': 'is_fraud'})

cards_df = cards_df.rename(columns={'id':'card_id'})

users_df = users_df.rename(columns={'id':'client_id'})

transactions_df = transactions_df.rename(columns={'mcc': 'mcc_code'})
transactions_df = transactions_df.rename(columns={'id': 'transaction_id'})




## 01-3_變數型態統一及缺失值處理

In [3]:
def add_missing_flags(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    """
    在 DataFrame 中對指定欄位建立 missing flag 欄位
    flag=1 表示缺失值，flag=0 表示非缺失值
    
    參數
    ----
    df : pd.DataFrame
        輸入的資料框
    cols : list
        要檢查的欄位名稱清單
    
    回傳
    ----
    pd.DataFrame : 新的資料框 (含新增的 flag 欄位)
    """
    for col in cols:
        df[f"{col}_missing_flag"] = df[col].isna().astype(int)
    return df

transactions_df = add_missing_flags(transactions_df, ["merchant_state", "zip", "errors"])

In [4]:
##train_fraud_labels_df##
train_fraud_labels_df["is_fraud"]=train_fraud_labels_df["is_fraud"].astype("category") 
train_fraud_labels_df["transactions_id"]=train_fraud_labels_df["transactions_id"].astype(int) #合併資料需要

##cards_df##
cards_df["card_brand"]=cards_df["card_brand"].astype("category") 
cards_df["card_type"]=cards_df["card_type"].astype("category")
#####不要load這行 cards_df["expires"]=pd.to_datetime(cards_df["expires"], format="%m/%Y")
cards_df["expires"] = pd.to_datetime(cards_df["expires"], format="%m/%Y").dt.to_period("M")
cards_df["has_chip"]=cards_df["has_chip"].astype("category")

cards_df['credit_limit'] = cards_df['credit_limit'].replace(r'[\$,]', '', regex=True).astype(int)
#####不要load這行 cards_df["acct_open_date"]=pd.to_datetime(cards_df["acct_open_date"], format="%m/%Y")
cards_df["acct_open_date"] = pd.to_datetime(cards_df["acct_open_date"], format="%m/%Y").dt.to_period("M")
#####不要load這行 cards_df["year_pin_last_changed"]=pd.to_datetime(cards_df["year_pin_last_changed"], format="%Y")
cards_df["year_pin_last_changed"] = pd.to_datetime(cards_df["year_pin_last_changed"], format="%Y").dt.to_period("Y")
cards_df["card_on_dark_web"]=cards_df["card_on_dark_web"].astype("category") 

##users_df##
users_df["birth_year"] = pd.to_datetime(users_df["birth_year"], format="%Y").dt.to_period("Y")
users_df["birth_month"] = pd.to_datetime(users_df["birth_month"], format="%m").dt.to_period("M")
users_df["gender"]=users_df["gender"].astype("category") 
users_df['per_capita_income'] = users_df['per_capita_income'].replace(r'[\$,]', '', regex=True).astype(int)
users_df['yearly_income'] = users_df['yearly_income'].replace(r'[\$,]', '', regex=True).astype(int)
users_df['total_debt'] = users_df['total_debt'].replace(r'[\$,]', '', regex=True).astype(int)

##transactions_df##
transactions_df["date"] = pd.to_datetime(transactions_df["date"])
#浮點數轉整數原因確定？
transactions_df['amount'] = transactions_df['amount'].replace(r'[\$,]', '', regex=True).astype(float).astype(int)
##負數取log調成1
#transactions_df['amount'] = transactions_df['amount'].replace(r'[\$,]', '', regex=True).astype(float)

transactions_df["use_chip"]=transactions_df["use_chip"].astype("category") 

transactions_df.loc[
    transactions_df['merchant_city'].str.lower() == 'online',
    'merchant_state'
] = 'online'

transactions_df.loc[
    transactions_df['merchant_city'].str.lower() == 'online',
    'zip'
] = 20000 #原本是-1
## 我沒有全部改，這樣完之後仍有89006筆Missing，剩下都是在國外
transactions_df['zip'] = transactions_df['zip'].fillna(10000) #原本是-999
transactions_df["zip"]=transactions_df["zip"].astype("int64")

transactions_df['errors'] = transactions_df['errors'].astype('category')
transactions_df['errors'] = transactions_df['errors'].cat.add_categories('No_error').fillna('No_error')



In [5]:
#cars one hot encoding
##統一類別變數轉dummy variable(要注意共線性問題，應刪掉其中之一)

#card_type 原始種類：Debit_57%, Credit_33%, Debit(Prepaid)_9%
#card_brand 原始種類：MasterCard_52%, Visa_38%, Amex_7%, Discovery_3%
#has_chip 原始種類：Yes_89%, No_11%
#card_on_dark_web 原始種類：No_0%
cols_to_encode = ['card_type', 'card_brand', 'has_chip']
cards_df[cols_to_encode] = cards_df[cols_to_encode].astype('category')
dummies_cards = pd.get_dummies(
    cards_df[cols_to_encode], 
    prefix=cols_to_encode, 
    dtype='uint8'
    )
cards_df = pd.concat([cards_df, dummies_cards], axis=1)

#use_chip 原始種類：Swiped_52%, Chipe_36%, Online_12%
dummies_use = pd.get_dummies(transactions_df['use_chip'], prefix='use_chip', dtype='uint8')
transactions_df = pd.concat([transactions_df, dummies_use], axis=1)

#gender 原始種類：Female_51%, Male_49%
dummies_gender = pd.get_dummies(users_df['gender'], prefix='gender', dtype='uint8')
users_df = pd.concat([users_df, dummies_gender], axis=1)


cards_df.drop(columns=["has_chip_NO","has_chip"], inplace=True)
transactions_df.drop(columns=["use_chip"], inplace=True)
users_df.drop(columns=["gender_Female"], inplace=True)

# 02_資料整併成一張dataframe

## 02-1_資料整併

In [6]:
#transactions_df.loc[transactions_df["transaction_id"] == 10649266] #transaction_id vs id

#原始資料筆數：13305915
### transactions_df+train_fraud_labels_df      left 會有4390952 missing values
merged = pd.merge(transactions_df, train_fraud_labels_df, left_on="transaction_id", right_on="transactions_id", how="outer")
### transactions_df train_fraud_labels_df(8914963) + users_df 對過去不會有missing values
merged = pd.merge(merged,users_df , left_on="client_id", right_on="client_id", how="left")
### transactions_df train_fraud_labels_df users_df + cards_df 對過去不會有missing values
merged = pd.merge(merged,cards_df , left_on="card_id", right_on="card_id", how="left")

#刪掉重複的columns
merged.drop(columns=["transactions_id"], inplace=True)
merged.drop(columns=["client_id_y"], inplace=True)

## 合併完之後最後處理is_fraud(原會有missing values問題)
merged["is_fraud"] = merged["is_fraud"].astype(str)
merged.loc[merged['is_fraud'].str.lower() == 'no','is_fraud'] = '0'
merged.loc[merged['is_fraud'].str.lower() == 'yes','is_fraud'] = '1'
merged["is_fraud"] = pd.to_numeric(merged["is_fraud"], errors="coerce").astype("Int64")

merged = add_missing_flags(merged, ["is_fraud"])

#merged.to_csv("merged.csv", index=False)

# 先刪除不需要的DataFrame以節省記憶體
del transactions_df, users_df, cards_df, train_fraud_labels_df, cols_to_encode, dummies_cards, dummies_use, dummies_gender

In [7]:
backup_merged = merged.copy()
#merged = backup_merged.copy()

# 04_RFM features engineering model

## 04-1_資料進行變數轉換以求模型配飾更佳表現

In [ ]:
##有出事再趕快回復原狀
merged = backup_merged.copy()

In [8]:
import numpy as np
import pandas as pd

# 確保日期是 datetime 並排序
merged['date'] = pd.to_datetime(merged['date'])
merged = merged.sort_values(by=['client_id_x', 'date']).reset_index(drop=True)

# --- RecencyInterval ---
merged['RecencyInterval'] = merged.groupby('client_id_x')['date'].diff().dt.total_seconds().fillna(0)/60

# --- TxnFrequency for multiple windows (向量化滑動窗口) ---
window_days = [7, 30, 60, 90]
for w in window_days:
    merged[f'TxnFrequency_{w}d'] = 0

def compute_freq_vectorized(dates, windows):
    """向量化計算每筆交易在每個 window 內的交易數"""
    n = len(dates)
    dates_int = dates.values.astype('datetime64[D]').astype(int)
    res = {w: np.zeros(n, dtype=int) for w in windows}
    for w in windows:
        left = 0
        counts = np.zeros(n, dtype=int)
        for right in range(n):
            while dates_int[right] - dates_int[left] > w:
                left += 1
            counts[right] = right - left + 1
        res[w] = counts
    return res

# 分組計算
for cid, g in merged.groupby('client_id_x', sort=False):
    freq_dict = compute_freq_vectorized(g['date'], window_days)
    for w in window_days:
        merged.loc[g.index, f'TxnFrequency_{w}d'] = freq_dict[w]

# --- AmtDelta ---
merged['prev_amount'] = merged.groupby('client_id_x')['amount'].shift(1)
merged['AmtDelta'] = merged['amount'] - merged['prev_amount']
merged['AmtDelta'] = merged['AmtDelta'].fillna(0)
merged.drop(columns='prev_amount', inplace=True)

In [9]:
# US region mapping
us_region_map = {
    'Northeast': ['NY','NJ','PA','MA','CT','RI','NH','VT','ME'],
    'Midwest': ['IL','OH','MI','IN','WI','MN','IA','MO','ND','SD','NE','KS'],
    'South': ['FL','GA','SC','NC','AL','MS','LA','TX','OK','TN','KY','VA','WV','AR','MD','DE','DC'],
    'West': ['CA','WA','OR','NV','AZ','NM','CO','UT','ID','MT','WY','AK','HI'],
}
continent_map = {
    'Europe': [ ... ],  # 原本 continent_map['Europe'] 可直接使用
    'Online': ['online','AA']
}

us_region_lookup = {state: region for region, states in us_region_map.items() for state in states}

# --- 向量化 location 特徵 ---
merged['merchant_online'] = merged['merchant_state'].eq('online').astype('uint8')
merged['merchant_us'] = merged['merchant_state'].isin(us_region_lookup.keys()).astype('uint8')
merged['merchant_eu'] = merged['merchant_state'].isin(continent_map['Europe']).astype('uint8')
merged['merchant_others'] = (~merged[['merchant_online','merchant_us','merchant_eu']].any(axis=1)).astype('uint8')


In [10]:
# --- 首次交易標記 ---
merged['FirstTxnInRegion'] = (~merged.duplicated(subset=['client_id_x', 'merchant_state'])).astype('uint8')

In [11]:
# DifferentState
merged['prev_state']=(merged
                     .groupby('client_id_x')['merchant_state']
                     .shift(1))

merged['DifferentState'] = (
    (merged['merchant_state'] != merged['prev_state'])
    & merged['prev_state'].notna()
).astype(int)

merged = merged.drop(columns=['prev_state'])


In [12]:
#merged[["card_id","card_number"]]
import numpy as np
from scipy import stats 

# === (1) log轉換 ===
merged['amount'] = np.where(merged['amount'] < 0, 0, merged['amount'])  # 負數變 0
merged['amount'] = np.log(merged['amount'] + 1)  

# === (3) 平方根轉換 ===
merged['credit_limit']=np.sqrt(merged['credit_limit'])
merged['total_debt']=np.sqrt(merged['total_debt'])

# === (3) 立方根轉換 ===
merged['yearly_income']=np.cbrt(merged['yearly_income'])
merged['per_capita_income']=np.cbrt(merged['per_capita_income'])

## Box-Cox Transformation
###merged['yearly_income'], fitted_lambda = stats.boxcox(merged['yearly_income'])

# === (5) Yeo–Johnson 轉換（可處理負值） ===
###merged['per_capita_income'], lambdaValue =stats.yeojohnson(merged['per_capita_income'])

## 04-2_分割訓練集及測試集

In [13]:
# --- 選取數值型變數 ---
num_cols = merged.select_dtypes(include=['int64', 'float64','uint8','datetime64[ns]']).columns
df2 = merged[num_cols]

# --- dropna ---
df_cleaned = df2.dropna()
del df2

# --- 避免共線性 ---
df_cleaned.drop(columns=["is_fraud_missing_flag","card_type_Debit (Prepaid)", 
                         "card_brand_Discover", "use_chip_Online Transaction"], inplace=True)

# --- 確保 date 欄位在 df_cleaned 中 ---
if 'date' not in df_cleaned.columns:
    df_cleaned['date'] = merged.loc[df_cleaned.index, 'date']

# --- 依時間排序 ---
df_sorted = df_cleaned.sort_values('date')
df_sorted['year'] = df_sorted['date'].dt.year

# 2010–2011 → block 0
# 2012–2013 → block 1
# ...
# 2018–2019 → block 4（如果你真的是 2010–2019 共 10 年，會有 5 個 block）
df_sorted['time_block'] = (df_sorted['year'] - 2010) // 2


#對每個時間block做切割
train_list = []
test_list = []

for block_id, block_df in df_sorted.groupby('time_block'):
    print(f"\nProcessing Block {block_id}")

    block_df = block_df.sort_values('date')
    split_index = int(len(block_df) * 0.8)

    train_block = block_df.iloc[:split_index].copy()
    test_block  = block_df.iloc[split_index:].copy()

    # -----------------------------
    # 1️⃣ 用「該 block 的 train」算 fraud rate
    # -----------------------------
    fraud_rate = (
        train_block
        .groupby('mcc_code')['is_fraud']
        .mean()
    )

    high_risk_mcc = fraud_rate[fraud_rate > 0.02].index

    # -----------------------------
    # 2️⃣ 套用到該 block 的 train / test
    # -----------------------------
    train_block['HighRiskMCC'] = train_block['mcc_code'].isin(high_risk_mcc).astype('uint8')
    test_block['HighRiskMCC']  = test_block['mcc_code'].isin(high_risk_mcc).astype('uint8')

    train_list.append(train_block)
    test_list.append(test_block)


train_df = pd.concat(train_list).drop(columns=['date', 'year', 'time_block'])
test_df  = pd.concat(test_list).drop(columns=['date', 'year', 'time_block'])


#比例檢查
for block_id in sorted(df_sorted['time_block'].unique()):
    print(f"\nBlock {block_id}")
    print("Train fraud count:")
    print(train_df.loc[df_sorted['time_block'] == block_id, 'is_fraud'].value_counts())
    print("Test fraud count:")
    print(test_df.loc[df_sorted['time_block'] == block_id, 'is_fraud'].value_counts())





Processing Block 0

Processing Block 1

Processing Block 2

Processing Block 3

Processing Block 4

Block 0
Train fraud count:
is_fraud
0    1353355
1       2610
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    338992
Name: count, dtype: Int64

Block 1
Train fraud count:
is_fraud
0    1433028
1       1152
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    357437
1      1108
Name: count, dtype: Int64

Block 2
Train fraud count:
is_fraud
0    1474582
1       1655
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    367862
1      1198
Name: count, dtype: Int64

Block 3
Train fraud count:
is_fraud
0    1493588
1       2448
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    373838
1       172
Name: count, dtype: Int64

Block 4
Train fraud count:
is_fraud
0    1367157
1       2393
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    341792
1       596
Name: count, dtype: Int64


In [14]:
'''
result = pd.DataFrame(columns=[
    "Model", "Features", 
    "Train AUC", "Test AUC", 
    "Train PR AUC", "Test PR AUC"
])

'''

# ALL features

all_cols = ['transaction_id', 'date', 'client_id_x', 'card_id', 'amount',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'mcc_code',
       'errors', 'merchant_state_missing_flag', 'zip_missing_flag',
       'errors_missing_flag', 'use_chip_Chip Transaction',
       'use_chip_Online Transaction', 'use_chip_Swipe Transaction',
       'current_age', 'retirement_age', 'birth_year', 'birth_month', 'gender',
       'address', 'latitude', 'longitude', 'per_capita_income',
       'yearly_income', 'total_debt', 'credit_score', 'num_credit_cards',
       'gender_Male', 'card_brand', 'card_type', 'card_number', 'expires',
       'cvv', 'num_cards_issued', 'credit_limit', 'acct_open_date',
       'year_pin_last_changed', 'card_on_dark_web', 'card_type_Credit',
       'card_type_Debit', 'card_type_Debit (Prepaid)', 'card_brand_Amex',
       'card_brand_Discover', 'card_brand_Mastercard', 'card_brand_Visa',
       'has_chip_YES', 'is_fraud_missing_flag']

VIF_col = ["is_fraud_missing_flag","card_type_Debit (Prepaid)", 
                         "card_brand_Discover", "use_chip_Online Transaction",'is_fraud_missing_flag','merchant_state_missing_flag', 'zip_missing_flag','card_on_dark_web']

identifier = [
    'transaction_id',
    'client_id_x',
    'card_id',
    'card_number',
    'cvv'
]

set_VIF = set(VIF_col)
set_identifier =set(identifier)
exclude_cols = set(VIF_col) | set(identifier)
all_cols = [x for x in all_cols if x not in exclude_cols]


# RFM features
rfm_cols = [
    'RecencyInterval', 'TxnFrequency_7d','TxnFrequency_30d',
    'TxnFrequency_60d', 'TxnFrequency_90d','AmtDelta'
]

# DK features
dk_cols = [
    'merchant_online', 'merchant_us', 'merchant_eu', 'merchant_others',
    'FirstTxnInRegion','HighRiskMCC','DifferentState'
]

# Grouping
feature_groups = {
    "X_all": all_cols,
    "X_rfm": rfm_cols,
    "X_dk": dk_cols,
    "X_all + X_rfm": all_cols + rfm_cols,
    "X_all + X_dk": all_cols + dk_cols,
    "X_rfm + X_dk": rfm_cols + dk_cols,
    "X_all + X_rfm + X_dk": all_cols + rfm_cols + dk_cols
}

## 04-3 老師建議我們先省略 Assumption: Avoid Multicollinearity

In [13]:
##處理共線性
train_df.drop(columns=["per_capita_income"], inplace=True)
train_df.drop(columns=["use_chip_Chip Transaction","merchant_state_missing_flag","zip_missing_flag"], inplace=True)           
train_df.drop(columns=["card_brand_Visa" ,"card_brand_Amex","card_type_Credit"], inplace=True)
#再重跑一次VIF

In [14]:
test_df.drop(columns=["per_capita_income"], inplace=True)
test_df.drop(columns=["use_chip_Chip Transaction","merchant_state_missing_flag","zip_missing_flag"], inplace=True)           
test_df.drop(columns=["card_brand_Visa" ,"card_brand_Amex","card_type_Credit"], inplace=True)

## 04-stepwise selection

In [24]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix
import warnings
import copy

# ===========================================================
# 1. 設定警告格式 (只顯示重點，不顯示路徑)
# ===========================================================
def short_warning(message, category, filename, lineno, file=None, line=None):
    return f"⚠️ {category.__name__}: {message}\n"

warnings.formatwarning = short_warning
# 若覺得太吵，可以取消註解下面這行，讓同類型警告只顯示一次
# warnings.simplefilter('once', category=RuntimeWarning)


# ===========================================================
# 2. Stepwise Logistic Regression (修正版)
# ===========================================================
def stepwise_logit_with_k_tables(
    train_df,
    test_df,
    dep_var="is_fraud",
    k=314657018,
    threshold_in=0.05,
    threshold_out=0.10,
    verbose=True
):
    y_train = train_df[dep_var]
    X_train = train_df.drop(columns=[dep_var])
    y_test = test_df[dep_var]
    X_test = test_df.drop(columns=[dep_var])

    included = []
    step = 0
    full_mode = (k == 314657018)

    # --- K=0 (Intercept Only) ---
    if k == 0:
        X_const = sm.add_constant(np.ones(len(y_train)), has_constant="add")
        final_model = sm.Logit(y_train, X_const).fit(disp=False)
        
        # 建立回傳格式
        ll_full = final_model.llf
        overall_fit = pd.DataFrame({
            "Measure": ["-2 Log Likelihood"], "Value": [round(-2 * ll_full, 3)]
        })
        coef_df = pd.DataFrame({
            "Independent Variable": ["const"],
            "B": final_model.params.values,
            "Exp(B)": np.exp(final_model.params.values)
        })
        not_in_eq_df = pd.DataFrame() # 空的
        
        if verbose: print("✅ Stepwise completed (Intercept only).")
        return overall_fit, coef_df, not_in_eq_df, final_model

    # --- Regular Stepwise ---
    while True:
        step += 1
        changed = False

        # Forward
        excluded = list(set(X_train.columns) - set(included))
        new_pvals = pd.Series(index=excluded, dtype=float)

        for new_var in excluded:
            try:
                X_try = sm.add_constant(X_train[included + [new_var]], has_constant="add")
                # 這裡不使用 catch_warnings，讓縮短後的警告顯示出來
                model = sm.Logit(y_train, X_try).fit(disp=False)
                new_pvals[new_var] = model.pvalues[new_var]
            except Exception:
                new_pvals[new_var] = np.nan

        if new_pvals.empty:
            break

        best_pval = new_pvals.min()
        if best_pval < threshold_in:
            best_var = new_pvals.idxmin()
            included.append(best_var)
            changed = True
            if verbose:
                print(f"🟢 Step {step}: Forward add {best_var} (p={best_pval:.4g})")

        # Backward
        if included:
            X_curr = sm.add_constant(X_train[included], has_constant="add")
            model = sm.Logit(y_train, X_curr).fit(disp=False)
            pvalues = model.pvalues.drop("const", errors="ignore")

            if not pvalues.empty:
                worst_pval = pvalues.max()
                if worst_pval > threshold_out:
                    worst_var = pvalues.idxmax()
                    included.remove(worst_var)
                    changed = True
                    if verbose:
                        print(f"🔴 Step {step}: Backward remove {worst_var} (p={worst_pval:.4g})")

        if not changed:
            if verbose: print(f"⚪ Step {step}: No change — stop.")
            break

        if not full_mode and len(included) >= k:
            if verbose: print(f"🟡 Reached k={k}, stop.")
            break

    # --- Final Model ---
    if len(included) == 0:
        X_final = sm.add_constant(np.ones(len(y_train)), has_constant="add")
    else:
        X_final = sm.add_constant(X_train[included], has_constant="add")

    try:
        final_model = sm.Logit(y_train, X_final).fit(disp=False)
    except ValueError:
        if verbose: print("⚠️ Final model failed. Fallback to intercept-only.")
        X_const = sm.add_constant(np.ones(len(y_train)), has_constant="add")
        final_model = sm.Logit(y_train, X_const).fit(disp=False)
        included = []

    # --- Report Generation (Simplified for brevity) ---
    ll_full = final_model.llf
    ll_null = sm.Logit(y_train, sm.add_constant(np.ones(len(y_train)), has_constant="add")).fit(disp=False).llf
    
    overall_fit = pd.DataFrame({
        "Measure": ["-2 Log Likelihood", "Pseudo R2"],
        "Value": [round(-2 * ll_full, 3), round(1 - (ll_full / ll_null), 3)]
    })

    coef_df = pd.DataFrame({
        "Independent Variable": final_model.params.index,
        "B": final_model.params.values,
        "Std. Error": final_model.bse.values,
        "Sig.": final_model.pvalues.values,
        "Exp(B)": np.exp(final_model.params.values)
    }).reset_index(drop=True)

    # 簡單建立 Not in Eq (省略 LRT 計算以加速)
    not_in_eq_df = pd.DataFrame({"Independent Variable": list(set(X_train.columns) - set(included))})

    if verbose:
        print(f"\n✅ Stepwise completed with {len(included)} variables")

    return overall_fit, coef_df, not_in_eq_df, final_model


# ===========================================================
# 3. Classification Table
# ===========================================================
def classification_table(model, df, target_col="is_fraud", cutoff=0.0015):
    y_true = df[target_col].astype(int)
    features = model.params.index.drop("const", errors="ignore")
    
    if len(features) == 0:
        X = sm.add_constant(np.ones(len(df)), has_constant="add")
    else:
        X = sm.add_constant(df[features], has_constant="add")

    y_pred_prob = model.predict(X)
    y_pred = (y_pred_prob >= cutoff).astype(int)
    
    cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
    TP, FN, FP, TN = cm.ravel()
    
    return pd.DataFrame({
        "Type": ["Fraud(1)", "Normal(0)", "Total"],
        "Correct": [TP, TN, TP+TN],
        "Total": [TP+FN, FP+TN, len(df)],
        "Accuracy (%)": [
            round(TP/(TP+FN)*100, 1) if (TP+FN)>0 else 0,
            round(TN/(FP+TN)*100, 1) if (FP+TN)>0 else 0,
            round((TP+TN)/len(df)*100, 1)
        ]
    })


# ===========================================================
# 4. Feature Selector (防呆修正版)
# ===========================================================
def select_features(df, feature_list, dep_var="is_fraud"):
    # 1. 確保 dep_var 不在特徵列表中
    features_to_use = [f for f in feature_list if f != dep_var and f in df.columns]
    
    # 2. 選取 特徵 + 目標
    final_cols = features_to_use + [dep_var]
    
    # 3. 建立子集
    df_sub = df[final_cols].copy()
    
    # 4. 轉數值
    df_sub = df_sub.select_dtypes(include=[np.number])
    df_sub = df_sub.astype(float)

    return df_sub


# ===========================================================
# 5. 定義特徵群組 (Feature Groups)
# ===========================================================
# 假設 all_cols, rfm_cols, dk_cols 已經在之前的步驟定義好了
# 這裡做最後的防呆清洗

identifier = ['transaction_id', 'client_id_x', 'card_id', 'card_number', 'cvv']
# 確保 all_cols 裡面沒有 ID 和 目標變數
all_cols = [x for x in all_cols if x not in identifier and x != 'is_fraud']

feature_groups = {
    "X_all": all_cols,
    "X_rfm": rfm_cols,
    "X_dk": dk_cols,
    "X_all + X_rfm": all_cols + rfm_cols,
    "X_all + X_dk": all_cols + dk_cols,
    "X_rfm + X_dk": rfm_cols + dk_cols,
    "X_all + X_rfm + X_dk": all_cols + rfm_cols + dk_cols
}

# 備份一份，以免修改到原始設定
current_feature_groups = copy.deepcopy(feature_groups)


# ===========================================================
# 6. 主程式迴圈 (Main Loop)
# ===========================================================
results = {}

for block_id, block_df in df_sorted.groupby('time_block'):

    print(f"\n{'='*40}")
    print(f"🚀 Running Block {block_id} (Year: {2010 + block_id*2} - {2011 + block_id*2})")
    print(f"{'='*40}")

    # 1. 時間排序與切分
    block_df = block_df.sort_values('date')
    split_index = int(len(block_df) * 0.8)
    
    train_raw = block_df.iloc[:split_index].copy()
    test_raw  = block_df.iloc[split_index:].copy()

    # 2. 計算 HighRiskMCC (防止 Data Leakage)
    fraud_rate = train_raw.groupby('mcc_code')['is_fraud'].mean()
    high_risk_mcc = fraud_rate[fraud_rate > 0.02].index
    
    train_raw['HighRiskMCC'] = train_raw['mcc_code'].isin(high_risk_mcc).astype('uint8')
    test_raw['HighRiskMCC']  = test_raw['mcc_code'].isin(high_risk_mcc).astype('uint8')

    # 初始化結果存放
    results[block_id] = {}

    # 3. 執行 Feature Groups
    for group_name, feature_list in current_feature_groups.items():

        print(f"\n🔹 Group: {group_name}")

        # 檢查欄位是否存在
        valid_features = [f for f in feature_list if f in train_raw.columns]
        if not valid_features:
            print("      ❌ No valid features found. Skipping.")
            continue

        # 選取特徵 (已含防呆)
        train_df = select_features(train_raw, valid_features, dep_var="is_fraud")
        test_df  = select_features(test_raw, valid_features, dep_var="is_fraud")
        
        # 排除常數欄位 (避免 LinAlgError)
        std_check = train_df.std()
        constant_cols = std_check[std_check == 0].index.tolist()
        if constant_cols:
            print(f"      ⚠️ Dropping constant cols: {constant_cols}")
            train_df = train_df.drop(columns=constant_cols)
            test_df = test_df.drop(columns=constant_cols)

        # 執行 Stepwise
        try:
            overall_fit, coef_df, not_in_eq_df, final_model = stepwise_logit_with_k_tables(
                train_df,
                test_df,
                dep_var="is_fraud",
                k=314657018, 
                verbose=True  # ✅ 確保這裡開啟顯示
            )

            train_table = classification_table(final_model, train_df)
            test_table  = classification_table(final_model, test_df)

            results[block_id][group_name] = {
                "overall_fit": overall_fit,
                "coef_df": coef_df,
                "model": final_model,
                "train_table": train_table,
                "test_table": test_table
            }

        except Exception as e:
            print(f"      ❌ Error: {str(e)}")
            results[block_id][group_name] = {"error": str(e)}

print("\n🎉 All blocks processed!")


🚀 Running Block 0 (Year: 2010 - 2011)

🔹 Group: X_all
      ⚠️ Dropping constant cols: ['use_chip_Chip Transaction']


⚠️ ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals


🟢 Step 1: Forward add zip (p=0)


⚠️ ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals


🟢 Step 2: Forward add longitude (p=0)


⚠️ RuntimeWarning: overflow encountered in exp
⚠️ RuntimeWarning: divide by zero encountered in log


🟢 Step 3: Forward add merchant_state_missing_flag (p=0)


KeyboardInterrupt: 

In [ ]:
## 分析用！！

results[
  block_id
][
  "X_all + X_rfm + X_dk"
]["coef_df"]


## 04-3 all丟模型內

In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
import copy

# ===========================================================
# 1. Full Logistic Regression 訓練函數
# ===========================================================
def fit_full_logit(train_df, test_df, feature_cols, dep_var="is_fraud"):
    """
    針對給定的特徵列表進行全變數 Logistic Regression (L2 Penalty)
    """
    # 準備 X 和 y
    X_train = train_df[feature_cols]
    y_train = train_df[dep_var]

    X_test = test_df[feature_cols]
    y_test = test_df[dep_var]

    # 處理缺失值 (Sklearn 不接受 NaN)
    # 簡單策略：填補 0 或刪除 (這邊假設你前面的資料清洗已經處理完 NaN，若有殘留建議 fillna)
    X_train = X_train.fillna(0)
    X_test = X_test.fillna(0)

    # 1. 標準化 (Standardization) - 非常重要！
    # 必須用 Train 的數據來 fit，然後 transform 到 Train 和 Test
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 2. 訓練模型 (使用 L2 Regularization 防止過擬合)
    model = LogisticRegression(
        penalty="l2",
        solver="lbfgs",
        max_iter=2000,  # 增加迭代次數避免收斂失敗
        n_jobs=-1,      # 平行運算
        random_state=42
    )
    
    try:
        model.fit(X_train_scaled, y_train)
        
        # 3. 預測機率
        train_pred_prob = model.predict_proba(X_train_scaled)[:, 1]
        test_pred_prob = model.predict_proba(X_test_scaled)[:, 1]

        # 4. 計算指標
        train_auc = roc_auc_score(y_train, train_pred_prob)
        test_auc = roc_auc_score(y_test, test_pred_prob)
        
        train_prauc = average_precision_score(y_train, train_pred_prob)
        test_prauc = average_precision_score(y_test, test_pred_prob)
        
        return {
            "status": "success",
            "model": model,
            "scaler": scaler,
            "metrics": {
                "Train AUC": round(train_auc, 4),
                "Test AUC": round(test_auc, 4),
                "Train PR-AUC": round(train_prauc, 4),
                "Test PR-AUC": round(test_prauc, 4)
            }
        }
        
    except Exception as e:
        return {"status": "error", "message": str(e)}


# ===========================================================
# 2. 準備 Feature Groups (確保無目標變數與ID)
# ===========================================================
# 確保 all_cols, rfm_cols, dk_cols 已經定義
identifier = ['transaction_id', 'client_id_x', 'card_id', 'card_number', 'cvv']
exclude_cols = set(identifier) | {'is_fraud', 'date', 'year', 'time_block'}

# 這裡假設你的 cols 列表已經存在，做最後一次清洗確保安全
# 如果你之前的 all_cols 已經清乾淨了，這裡只是雙重保險
def clean_cols(cols):
    return [c for c in cols if c not in exclude_cols]

feature_groups = {
    "X_all": clean_cols(all_cols),
    "X_rfm": clean_cols(rfm_cols),
    "X_dk": clean_cols(dk_cols),
    "X_all + X_rfm": clean_cols(all_cols + rfm_cols),
    "X_all + X_dk": clean_cols(all_cols + dk_cols),
    "X_rfm + X_dk": clean_cols(rfm_cols + dk_cols),
    "X_all + X_rfm + X_dk": clean_cols(all_cols + rfm_cols + dk_cols)
}


# ===========================================================
# 3. 主迴圈 (Block + Feature Groups)
# ===========================================================
full_logit_results = {}

for block_id, block_df in df_sorted.groupby('time_block'):
    
    print(f"\n{'='*50}")
    print(f"🚀 Running Full Logit for Block {block_id} (Year: {2010 + block_id*2} - {2011 + block_id*2})")
    print(f"{'='*50}")

    # --- 資料切分 (Split) ---
    block_df = block_df.sort_values('date')
    split_index = int(len(block_df) * 0.8)
    
    train_raw = block_df.iloc[:split_index].copy()
    test_raw  = block_df.iloc[split_index:].copy()

    # --- 重算 HighRiskMCC (Anti-Leakage) ---
    fraud_rate = train_raw.groupby('mcc_code')['is_fraud'].mean()
    high_risk_mcc = fraud_rate[fraud_rate > 0.02].index
    
    train_raw['HighRiskMCC'] = train_raw['mcc_code'].isin(high_risk_mcc).astype('uint8')
    test_raw['HighRiskMCC']  = test_raw['mcc_code'].isin(high_risk_mcc).astype('uint8')

    # 初始化結果存放
    full_logit_results[block_id] = {}

    # --- 針對每個 Feature Group 跑模型 ---
    for group_name, feature_list in feature_groups.items():
        
        # 確保特徵存在於 DataFrame
        valid_features = [f for f in feature_list if f in train_raw.columns]
        
        # 簡單檢查：若特徵少於 1 個就跳過
        if not valid_features:
            print(f"   ⚠️ Group: {group_name} - No valid features found. Skipping.")
            continue
            
        print(f"   🔹 Group: {group_name} ({len(valid_features)} features)...", end=" ")

        # 執行 Full Logistic Regression
        res = fit_full_logit(
            train_raw, 
            test_raw, 
            feature_cols=valid_features, 
            dep_var="is_fraud"
        )

        if res["status"] == "success":
            metrics = res["metrics"]
            print(f"✅ Done.")
            print(f"      Train AUC: {metrics['Train AUC']} | Test AUC: {metrics['Test AUC']}")
            print(f"      Train PR : {metrics['Train PR-AUC']} | Test PR : {metrics['Test PR-AUC']}")
            
            # 儲存結果
            full_logit_results[block_id][group_name] = metrics
        else:
            print(f"❌ Error: {res['message']}")
            full_logit_results[block_id][group_name] = {"error": res['message']}

print("\n🎉 All Full Logistic Regression models completed!")


🚀 Running Full Logit for Block 0 (Year: 2010 - 2011)
   🔹 Group: X_all (25 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done.
      Train AUC: 0.9526 | Test AUC: nan
      Train PR : 0.0372 | Test PR : 0.0
   🔹 Group: X_rfm (6 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done.
      Train AUC: 0.7276 | Test AUC: nan
      Train PR : 0.0062 | Test PR : 0.0
   🔹 Group: X_dk (7 features)... 

...

✅ Done.
      Train AUC: 0.9769 | Test AUC: nan
      Train PR : 0.2107 | Test PR : 0.0
   🔹 Group: X_rfm + X_dk (13 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done.
      Train AUC: 0.9805 | Test AUC: nan
      Train PR : 0.2253 | Test PR : 0.0
   🔹 Group: X_all + X_rfm + X_dk (38 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done.
      Train AUC: 0.9826 | Test AUC: nan
      Train PR : 0.2445 | Test PR : 0.0

🚀 Running Full Logit for Block 1 (Year: 2012 - 2013)
   🔹 Group: X_all (25 features)... ✅ Done.
      Train AUC: 0.8542 | Test AUC: 0.9019
      Train PR : 0.0084 | Test PR : 0.0338
   🔹 Group: X_rfm (6 features)... ✅ Done.
      Train AUC: 0.6996 | Test AUC: 0.7287
      Train PR : 0.0024 | Test PR : 0.0138
   🔹 Group: X_dk (7 features)... ✅ Done.
      Train AUC: 0.814 | Test AUC: 0.8753
      Train PR : 0.021 | Test PR : 0.0801
   🔹 Group: X_all + X_rfm (31 features)... ✅ Done.
      Train AUC: 0.887 | Test AUC: 0.9293
      Train PR : 0.0143 | Test PR : 0.0644
   🔹 Group: X_all + X_dk (32 features)... ✅ Done.
      Train AUC: 0.8458 | Test AUC: 0.8936
      Train PR : 0.0279 | Test PR : 0.0885
   🔹 Group: X_rfm + X_dk (13 features)... ✅ Done.
      Train AUC: 0.8738 | Test AUC: 0.921
      Train PR : 0.0432 | Test PR : 0.1773
   🔹 Group: X_all + X_rfm + X_dk (38 features)... ✅ Done.
      Train

In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# ==========================================
# 1. 將巢狀字典轉為 DataFrame
# ==========================================
data_list = []

for block_id, groups in full_logit_results.items():
    for group_name, metrics in groups.items():
        # 排除發生錯誤的組別
        if "error" in metrics:
            continue
            
        row = {
            "Block": block_id,
            "Year": f"{2010 + block_id*2}-{2011 + block_id*2}",
            "Feature Group": group_name,
            "Train AUC": metrics.get("Train AUC"),
            "Test AUC": metrics.get("Test AUC"),
            "Train PR-AUC": metrics.get("Train PR-AUC"),
            "Test PR-AUC": metrics.get("Test PR-AUC")
        }
        data_list.append(row)

df_results = pd.DataFrame(data_list)

# 調整欄位順序
cols = ["Block", "Year", "Feature Group", "Train AUC", "Test AUC", "Train PR-AUC", "Test PR-AUC"]
df_results = df_results[cols]

# 顯示前幾筆
print("=== 詳細結果總表 (前 5 筆) ===")
print(df_results.head())

# ==========================================
# 2. 製作 Test AUC 比較矩陣 (Pivot Table)
# ==========================================
# 這是最直觀的表格：橫軸是時間，縱軸是特徵群組，數值是 Test AUC
pivot_auc = df_results.pivot(index="Feature Group", columns="Year", values="Test PR-AUC")
pivot_trauc = df_results.pivot(index="Feature Group", columns="Year", values="Train PR-AUC")

print("\n=== Traing PR-AUC 績效矩陣 (數值越高越好) ===")
print(pivot_trauc)
print("\n=== Test PR-AUC 績效矩陣 (數值越高越好) ===")
print(pivot_auc)

=== 詳細結果總表 (前 5 筆) ===
   Block       Year  Feature Group  Train AUC  Test AUC  Train PR-AUC  \
0      0  2010-2011          X_all     0.9526       NaN        0.0372   
1      0  2010-2011          X_rfm     0.7276       NaN        0.0062   
2      0  2010-2011           X_dk     0.9704       NaN        0.1757   
3      0  2010-2011  X_all + X_rfm     0.9609       NaN        0.0643   
4      0  2010-2011   X_all + X_dk     0.9769       NaN        0.2107   

   Test PR-AUC  
0          0.0  
1          0.0  
2          0.0  
3          0.0  
4          0.0  

=== Traing PR-AUC 績效矩陣 (數值越高越好) ===
Year                  2010-2011  2012-2013  2014-2015  2016-2017  2018-2019
Feature Group                                                              
X_all                    0.0372     0.0084     0.0352     0.0385     0.0982
X_all + X_dk             0.2107     0.0279     0.0934     0.0725     0.6605
X_all + X_rfm            0.0643     0.0143     0.0827     0.0620     0.1242
X_all + X_rfm + X_d

## XGB

In [20]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, average_precision_score
import copy

# ===========================================================
# 1. 定義單次 XGBoost 訓練函數
# ===========================================================
def run_xgb_single(train_df, test_df, feature_cols, dep_var="is_fraud"):
    """
    針對給定的 Train/Test 和特徵列表訓練 XGBoost
    """
    X_train = train_df[feature_cols]
    y_train = train_df[dep_var]
    X_test = test_df[feature_cols]
    y_test = test_df[dep_var]

    # 初始化 XGBoost
    model = XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="auc", # 改用 AUC 作為評估指標通常比較直觀，logloss 也可以
        random_state=42,
        tree_method="hist", # 加速訓練
        n_jobs=-1,
        early_stopping_rounds=50 # 如果 50 輪內驗證集 AUC 沒提升就停止
    )

    try:
        # 訓練模型
        model.fit(
            X_train, y_train,
            eval_set=[(X_test, y_test)],
            verbose=False # 關閉詳細訓練過程輸出，避免洗版
        )

        # 預測機率
        train_pred = model.predict_proba(X_train)[:, 1]
        test_pred = model.predict_proba(X_test)[:, 1]

        # 計算指標
        train_roc = roc_auc_score(y_train, train_pred)
        test_roc = roc_auc_score(y_test, test_pred)
        train_pr = average_precision_score(y_train, train_pred)
        test_pr = average_precision_score(y_test, test_pred)

        # 取得最佳迭代次數 (如果觸發早停)
        best_iter = model.best_iteration if hasattr(model, 'best_iteration') else 300

        return {
            "status": "success",
            "model": model,
            "metrics": {
                "Train AUC": round(train_roc, 4),
                "Test AUC": round(test_roc, 4),
                "Train PR-AUC": round(train_pr, 4),
                "Test PR-AUC": round(test_pr, 4),
                "Best Iteration": best_iter
            }
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}


# ===========================================================
# 2. 準備 Feature Groups (確保無目標變數與ID)
# ===========================================================
# 假設 identifier, all_cols, rfm_cols, dk_cols 已經定義好
identifier = ['transaction_id', 'client_id_x', 'card_id', 'card_number', 'cvv']
exclude_cols = set(identifier) | {'is_fraud', 'date', 'year', 'time_block'}

def clean_cols(cols):
    return [c for c in cols if c not in exclude_cols]

feature_groups = {
    "X_all": clean_cols(all_cols),
    "X_rfm": clean_cols(rfm_cols),
    "X_dk": clean_cols(dk_cols),
    "X_all + X_rfm": clean_cols(all_cols + rfm_cols),
    "X_all + X_dk": clean_cols(all_cols + dk_cols),
    "X_rfm + X_dk": clean_cols(rfm_cols + dk_cols),
    "X_all + X_rfm + X_dk": clean_cols(all_cols + rfm_cols + dk_cols)
}

# ===========================================================
# 3. 主迴圈 (Block + Feature Groups)
# ===========================================================
xgb_results = {}

# 確保 df_sorted 存在
if 'df_sorted' not in locals():
    raise ValueError("df_sorted is not defined. Please ensure your DataFrame is prepared.")

for block_id, block_df in df_sorted.groupby('time_block'):
    
    print(f"\n{'='*60}")
    print(f"🚀 Running XGBoost for Block {block_id} (Year: {2010 + block_id*2} - {2011 + block_id*2})")
    print(f"{'='*60}")

    # --- 資料切分 (Split) ---
    block_df = block_df.sort_values('date')
    split_index = int(len(block_df) * 0.8)
    
    train_raw = block_df.iloc[:split_index].copy()
    test_raw  = block_df.iloc[split_index:].copy()

    # --- 重算 HighRiskMCC (Anti-Leakage) ---
    fraud_rate = train_raw.groupby('mcc_code')['is_fraud'].mean()
    high_risk_mcc = fraud_rate[fraud_rate > 0.02].index
    
    train_raw['HighRiskMCC'] = train_raw['mcc_code'].isin(high_risk_mcc).astype('uint8')
    test_raw['HighRiskMCC']  = test_raw['mcc_code'].isin(high_risk_mcc).astype('uint8')

    # 初始化結果存放
    xgb_results[block_id] = {}

    # --- 針對每個 Feature Group 跑模型 ---
    for group_name, feature_list in feature_groups.items():
        
        # 確保特徵存在於 DataFrame
        valid_features = [f for f in feature_list if f in train_raw.columns]
        
        # 簡單檢查
        if not valid_features:
            print(f"   ⚠️ Group: {group_name} - No valid features found. Skipping.")
            continue
            
        print(f"   🔹 Group: {group_name} ({len(valid_features)} features)...", end=" ")

        # 執行 XGBoost
        res = run_xgb_single(
            train_raw, 
            test_raw, 
            feature_cols=valid_features, 
            dep_var="is_fraud"
        )

        if res["status"] == "success":
            metrics = res["metrics"]
            print(f"✅ Done (Best Iter: {metrics['Best Iteration']})")
            print(f"      Train AUC: {metrics['Train AUC']} | Test AUC: {metrics['Test AUC']}")
            print(f"      Train PR : {metrics['Train PR-AUC']} | Test PR : {metrics['Test PR-AUC']}")
            
            xgb_results[block_id][group_name] = metrics
        else:
            print(f"❌ Error: {res['message']}")
            xgb_results[block_id][group_name] = {"error": res['message']}

print("\n🎉 All XGBoost models completed!")


🚀 Running XGBoost for Block 0 (Year: 2010 - 2011)
   🔹 Group: X_all (25 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/xgboost/callback.py:266: UserWarning: [16:14:53] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/xgboost/callback.py:266: UserWarning: [16:14:54] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.c

✅ Done (Best Iter: 0)
      Train AUC: 0.9878 | Test AUC: nan
      Train PR : 0.2217 | Test PR : 0.0
   🔹 Group: X_rfm (6 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/xgboost/callback.py:266: UserWarning: [16:14:55] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/xgboost/callback.py:266: UserWarning: [16:14:56] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.c

✅ Done (Best Iter: 0)
      Train AUC: 0.7702 | Test AUC: nan
      Train PR : 0.0084 | Test PR : 0.0
   🔹 Group: X_dk (7 features)... 

...

✅ Done (Best Iter: 0)
      Train AUC: 0.9946 | Test AUC: nan
      Train PR : 0.2525 | Test PR : 0.0
   🔹 Group: X_rfm + X_dk (13 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/xgboost/callback.py:266: UserWarning: [16:15:01] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done (Best Iter: 0)
      Train AUC: 0.9782 | Test AUC: nan
      Train PR : 0.071 | Test PR : 0.0
   🔹 Group: X_all + X_rfm + X_dk (38 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/xgboost/callback.py:266: UserWarning: [16:15:02] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/xgboost/callback.py:266: UserWarning: [16:15:03] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.c

✅ Done (Best Iter: 0)
      Train AUC: 0.9949 | Test AUC: nan
      Train PR : 0.2657 | Test PR : 0.0

🚀 Running XGBoost for Block 1 (Year: 2012 - 2013)
   🔹 Group: X_all (25 features)... ✅ Done (Best Iter: 66)
      Train AUC: 0.9832 | Test AUC: 0.9861
      Train PR : 0.6527 | Test PR : 0.7896
   🔹 Group: X_rfm (6 features)... ✅ Done (Best Iter: 43)
      Train AUC: 0.8576 | Test AUC: 0.7977
      Train PR : 0.0177 | Test PR : 0.0192
   🔹 Group: X_dk (7 features)... ✅ Done (Best Iter: 38)
      Train AUC: 0.8765 | Test AUC: 0.9387
      Train PR : 0.0213 | Test PR : 0.083
   🔹 Group: X_all + X_rfm (31 features)... ✅ Done (Best Iter: 118)
      Train AUC: 0.9954 | Test AUC: 0.992
      Train PR : 0.7646 | Test PR : 0.8077
   🔹 Group: X_all + X_dk (32 features)... ✅ Done (Best Iter: 52)
      Train AUC: 0.9763 | Test AUC: 0.9904
      Train PR : 0.6197 | Test PR : 0.7917
   🔹 Group: X_rfm + X_dk (13 features)... ✅ Done (Best Iter: 23)
      Train AUC: 0.9396 | Test AUC: 0.9664
      Tr

In [24]:
# 整理 XGBoost 結果表格
xgb_data_list = []

for block_id, groups in xgb_results.items():
    for group_name, metrics in groups.items():
        if "error" in metrics: continue
        row = {
            "Block": block_id,
            "Year": f"{2010 + block_id*2}-{2011 + block_id*2}",
            "Feature Group": group_name,
            "Train AUC": metrics.get("Train AUC"),
            "Test AUC": metrics.get("Test AUC"),
            "Train PR-AUC": metrics.get("Train PR-AUC"),
            "Test PR-AUC": metrics.get("Test PR-AUC"),
            "Best Iter": metrics.get("Best Iteration")
        }
        xgb_data_list.append(row)

df_xgb = pd.DataFrame(xgb_data_list)
print(df_xgb.head())

# 製作 Test AUC 矩陣
pivot_trxgb = df_xgb.pivot(index="Feature Group", columns="Year", values="Train PR-AUC")
pivot_testxgb = df_xgb.pivot(index="Feature Group", columns="Year", values="Test PR-AUC")
print("\n=== XGBoost Train PR-AUC Matrix ===")
print(pivot_trxgb)
print("\n")
print("\n=== XGBoost Test PR-AUC Matrix ===")
print(pivot_testxgb)

   Block       Year  Feature Group  Train AUC  Test AUC  Train PR-AUC  \
0      0  2010-2011          X_all     0.9878       NaN        0.2217   
1      0  2010-2011          X_rfm     0.7702       NaN        0.0084   
2      0  2010-2011           X_dk     0.9743       NaN        0.1308   
3      0  2010-2011  X_all + X_rfm     0.9793       NaN        0.0874   
4      0  2010-2011   X_all + X_dk     0.9946       NaN        0.2525   

   Test PR-AUC  Best Iter  
0          0.0          0  
1          0.0          0  
2          0.0          0  
3          0.0          0  
4          0.0          0  

=== XGBoost Train PR-AUC Matrix ===
Year                  2010-2011  2012-2013  2014-2015  2016-2017  2018-2019
Feature Group                                                              
X_all                    0.2217     0.6527     0.9058     0.3223     0.9393
X_all + X_dk             0.2525     0.6197     0.8607     0.5034     0.9234
X_all + X_rfm            0.0874     0.7646     0.968

In [17]:
cols_to_drop = [
    'transaction_id',
    'client_id_x',
    'card_id',
    'card_number',
    'cvv'
]

train_df = train_df.drop(columns=cols_to_drop, errors='ignore')
test_df  = test_df.drop(columns=cols_to_drop, errors='ignore')

In [ ]:
result = fit_full_logit(train_df, test_df, dep_var="is_fraud")

print("Train AUC:", result["train_auc"])
print("Test AUC:", result["test_auc"])

print("Train PR-AUC:", result["train_prauc"])
print("Test PR-AUC:", result["test_prauc"])


NameError: name 'fit_full_logit' is not defined

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score

def fit_full_logit(train_df, test_df, dep_var="is_fraud"):
    # 1. split
    X_train = train_df.drop(columns=[dep_var])
    y_train = train_df[dep_var]

    X_test = test_df.drop(columns=[dep_var])
    y_test = test_df[dep_var]

    # 🎯 確保 test 欄位順序 = train 欄位順序
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    # 2. 標準化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 3. Logistic Regression
    model = LogisticRegression(
        penalty="l2",
        solver="lbfgs",
        max_iter=1000,
        n_jobs=-1,
        class_weight="balanced"
    )
    model.fit(X_train_scaled, y_train)

    # 4. probabilities
    train_pred = model.predict_proba(X_train_scaled)[:, 1]
    test_pred = model.predict_proba(X_test_scaled)[:, 1]

    # 5. ROC AUC
    train_auc = roc_auc_score(y_train, train_pred)
    test_auc = roc_auc_score(y_test, test_pred)

    # 6. PR AUC
    train_prauc = average_precision_score(y_train, train_pred)
    test_prauc = average_precision_score(y_test, test_pred)

    return {
        "model": model,
        "scaler": scaler,
        "train_pred": train_pred,
        "test_pred": test_pred,
        "train_auc": train_auc,
        "test_auc": test_auc,
        "train_prauc": train_prauc,
        "test_prauc": test_prauc
    }


In [16]:
result = fit_full_logit(train_df, test_df, dep_var="is_fraud")

print("Train AUC:", result["train_auc"])
print("Test AUC:", result["test_auc"])

print("Train PR-AUC:", result["train_prauc"])
print("Test PR-AUC:", result["test_prauc"])

Train AUC: 0.8963198198876533
Test AUC: 0.8912327090965326
Train PR-AUC: 0.04883650297518295
Test PR-AUC: 0.03905783518019472
